In [5]:
from IPython.display import display
import requests
import urllib
import json
import pandas as pd
import numpy as np
from pandas import DataFrame
from bs4 import BeautifulSoup
import time
from datetime import datetime
from datetime import timedelta

### ANA Airline Crawling Job
- 홈페이지
- https://aswbe-i.ana.co.jp/international_asw/pages/revenue/search/roundtrip/search_roundtrip_input.xhtml?rand=20170727090147&CONNECTION_KIND=KOR&LANG=ko
 사이트 접속 조건 검색

In [10]:
def crawling_NH(dpt,arr,dpt_date):
    url = 'https://aswbe-i.ana.co.jp/international_asw/pages/revenue/search/roundtrip/search_roundtrip_input.xhtml?rand=20170727090147&CONNECTION_KIND=KOR&LANG=ko'
    head = {
        'Accept-Language':'ko-KR,ko',
        'Referer':'https://www.ana.co.jp/asw/wws/kr/k/',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    param = {
        'hiddenSearchMode':'ONE_WAY',
        'departureAirportCode:field':dpt,
        #'DepApoText':'Seoul(Incheon)/Seoul(Incheon)[ICN]',
        'arrivalAirportCode:field':arr,
        #'ArrApoText':'Tokyo(Narita)/Tokyo(Narita)[NRT]',
        'departureDate:field':dpt_date,
        'returnDate:field':dpt_date,
        'adultCount':'1',
        'youngAdultCount':'0',
        'childCount':'0',
        'infantCount':'0',
        'boardingClass':'ECD',
        'promotionCode':'',
        'STATIC':'STATIC',
    }
    req = requests.post(url,param,headers=head)
    return req.text
## 포맷 : 대상항공사코드,플라이트,출발일,출발지,도착지,출발시간,도착시간,가격,유류세,세금,좌석수
def scraping_NH(head,raw_data):
    start_txt = 'addRecommendation('
    end_txt = ');'
    pos = 0
    fare_tmp = []
    while raw_data.find(start_txt,pos) > 0:
        start_pos = raw_data.find(start_txt,pos) + len(start_txt)
        end_pos = raw_data.find(end_txt,start_pos)
        fare_tmp.append(raw_data[start_pos:end_pos].split(','))
        pos = end_pos
    #print(fare_tmp)
    ##필요한 데이터 추출
    fare_map = {}
    for fare in fare_tmp:
        ## 구분키
        fare_map[fare[3][1:-1]+':'+fare[1]] = fare[-7:-5]
    ## 스케줄 정보
    soup = BeautifulSoup(raw_data,'lxml')
    sch_list = []##  구분키, 출발시간, 도착시간, 편명
    for sch in soup.select('.oneWayDisplayPlan'):
        sch_line = [sch.select('i')[0]['data-value']]
        sch_line.extend([div.text.strip() for div in sch.select('.flightSchedule div.designTd.segTableCell')[0:5:2]])
        sch_list.append(sch_line)
    fare_list = []
    for sch in sch_list:
        line_list = [head['airline'],sch[3],head['dpt_date'],head['dpt'],head['arr'],sch[1],sch[2]]
        fare_info = fare_map[sch[0]]
        line_list.append(fare_info[0])
        line_list.append('0') ## 유류세, 세금이 합해진 형태로 세금 필드에 저장
        line_list.append(fare_info[1])
        line_list.append('9') ## 좌석 정보 없음
        fare_list.append(line_list)
    return fare_list

In [14]:
## 홈페이지
dpt = 'ICN'
arr = 'NRT'
dpt_date = '20170816'
raw_data = crawling_NH(dpt,arr,dpt_date)
head = {
    'airline':'NH',
    'dpt_date':dpt_date,
    'dpt':dpt,
    'arr':arr
}
fare_list = scraping_NH(head,raw_data)
pd.DataFrame(fare_list)

,0,1,2,3,4,5,6,7,8,9,10
0,NH,NH6972,20170816,ICN,NRT,09:00,11:10,368500.0,0,31500.0,9
1,NH,NH6974,20170816,ICN,NRT,10:10,12:20,368500.0,0,31500.0,9
2,NH,NH7046,20170816,ICN,NRT,12:35,14:55,368500.0,0,31500.0,9
3,NH,NH6978,20170816,ICN,NRT,18:50,21:10,368500.0,0,31500.0,9
4,NH,NH6964,20170816,ICN,NRT,09:00,10:50,368500.0,0,39200.0,9
5,NH,NH6952,20170816,ICN,NRT,07:55,09:40,368500.0,0,36000.0,9
6,NH,NH6956,20170816,ICN,NRT,19:10,20:55,368500.0,0,36000.0,9
7,NH,NH6888,20170816,ICN,NRT,17:20,19:05,368500.0,0,36000.0,9
